In [1]:
import numpy as np
import pandas as pd
import random as r
import math
import os
from datetime import timedelta
from datetime import datetime

import matplotlib.pyplot as plot
import networkx as nx
import seaborn as sns

from pyomo.environ import *
from pyomo.opt import SolverFactory
from gurobipy import GRB

from scipy.stats import beta

In [2]:
#id directories
data_dir = os.getcwd().replace('/model/not_modules', '/input_data/seattle')
results_dir = os.getcwd().replace('/model/not_modules', '/results')


reliability_levels = [0,1,2]
budget_levels = range(1,10+1)

level_amount_incrememented = 1000000

trade_off_dict = {}

In [3]:
#read data
os.chdir(data_dir)
items_df = pd.read_csv('items_df.csv')
items_df = items_df.dropna() #only include items without nan
incoming_orders_df = pd.read_csv('incoming_orders_df.csv')
incoming_orders_df['Date'] = pd.to_datetime(incoming_orders_df['Date'])

#get time since oldest order
oldest_order = min(incoming_orders_df['Date'])
today = datetime.now()
days_since_oldest_order = (today-oldest_order).days
weeks_since_oldest_order = math.floor(days_since_oldest_order/7)

#to estimate demand
demand_df = items_df[['ItemTypeID', 'DemandPerDay']].groupby('ItemTypeID').sum().reset_index()
demand_df['DemandPerWeek'] = round(demand_df['DemandPerDay']*7,0)

In [4]:
#define sets
K = items_df['ItemTypeID'].unique()
I = items_df['ItemID'].unique()
T = 10

In [5]:
def initialize_model(CAP_reliability, DEMAND_reliability, 
                     budget_level):
    
    global level_amount_incrememented
    global K
    global I
    global T

    #define parameters

    r_k_dict = {
    }

    rank_df = items_df[['ItemTypeID', 'ItemType', 'Essentiality']].\
    groupby(['ItemTypeID', 'ItemType'])['Essentiality'].max().reset_index()

    for k in rank_df['ItemTypeID']:
        temp = rank_df[rank_df['ItemTypeID']==k]

        r_k_dict[k] = temp['Essentiality'].values[0]

    #quality 
    q_k_i_dict = {
    }

    for i in items_df['ItemID']:
        temp = items_df[items_df['ItemID'] == i]
        k_temp = temp['ItemTypeID'].values[0]
        quality_temp = temp['Quality'].values[0]

        q_k_i_dict[tuple([k_temp,i])] = quality_temp

    #starting inventory
    z_k_i_init_dict = {
    }

    for i in items_df['ItemID']:
        temp = items_df[items_df['ItemID'] == i]
        k_temp = temp['ItemTypeID'].values[0]
        beg_inv_temp = temp['BeginningInventory'].values[0]

        z_k_i_init_dict[tuple([k_temp,i])] = beg_inv_temp

    alpha_k_init_dict = {
    }

    #for now assuming no open demand
    for k in rank_df['ItemTypeID']:
        alpha_k_init_dict[k] = 0

    #lead time
    f_k_i_tDiff_dict = {}

    for i in items_df['ItemID']:
        last_prob = 0
        temp = items_df[items_df['ItemID'] == i]
        max_delay = temp['max_delay'].values[0]
        min_delay = temp['min_delay'].values[0]
        alpha_temp = temp['alpha_delay'].values[0]
        beta_temp = temp['beta_delay'].values[0]
        k_temp = temp['ItemTypeID'].values[0]
        for t in range(T+weeks_since_oldest_order+1): #to make sure the prob distibution can calculate okit
            norm_temp = 0
            if (max_delay - min_delay) != 0:
                norm_temp = (t - min_delay)/(max_delay - min_delay)
            cum_prob_temp = beta.cdf(norm_temp, alpha_temp, beta_temp)
            prob_temp = cum_prob_temp - last_prob
            if np.isnan(prob_temp):
                f_k_i_tDiff_dict[tuple([k_temp, i, t])] = 0
            else:
                f_k_i_tDiff_dict[tuple([k_temp, i, t])] = prob_temp

            last_prob = cum_prob_temp

    o_k_i_t_dict = {}

    for k in K:
        for t in I:
            for t in range(1,T+1):
                o_k_i_t_dict[tuple([k,i,t])] = 0

    for index in incoming_orders_df.index:
        item_id = incoming_orders_df.ItemTypeID.iloc[index]
        ordered_date = pd.to_datetime(incoming_orders_df['Date']).iloc[index]
        days_since_order = (datetime.now() - pd.to_datetime(incoming_orders_df['Date'])\
                            .iloc[index]).days
        weeks_since_order = math.floor(days_since_order/7)
        amount_ordered = incoming_orders_df.PrimaryQty.iloc[index]

        for t in range(1,T+1):
            tDiff = t + weeks_since_order
            o_k_i_t_dict[tuple([k,i,t])] = f_k_i_tDiff_dict[tuple([k,i,tDiff])]*amount_ordered

    #cost
    c_k_i_dict = {
    }

    for i in items_df['ItemID']:
        temp = items_df[items_df['ItemID'] == i]
        k_temp = temp['ItemTypeID'].values[0]
        cost = temp['cost'].values[0]

        c_k_i_dict[tuple([k_temp,i])] = cost


    b = budget_level*level_amount_incrememented

    #capacity
    CAP_k_i_t_dict_mu_sd = {}

    for i in items_df['ItemID']:
        temp = items_df[items_df['ItemID'] == i]
        k_temp = int(temp['ItemTypeID'].values[0])
        capacity = round(temp['capacity'].values[0],0)
        for t in range(T+1):
            if k_temp == 2:
                CAP_k_i_t_dict_mu_sd[tuple([k_temp,i,t])] = [capacity*40, capacity*.1]
            else:
                CAP_k_i_t_dict_mu_sd[tuple([k_temp,i,t])] = [capacity, capacity*.1]

    #demand
    D_k_t_dict_mu_sd = {
    }

    for k in demand_df['ItemTypeID']:
        temp = demand_df[demand_df['ItemTypeID'] == k]
        demand_per_week = temp['DemandPerWeek'].values[0]

        for t in range(T+1):
            D_k_t_dict_mu_sd[tuple([k,t])] = [demand_per_week, demand_per_week*.1]

    model = ConcreteModel()

    #####define sets#######
    model.K = Set(initialize = K)
    model.I = Set(initialize = I)
    model.T = Set(initialize = range(1,T+1))
    model.T_beg = model.T|Set(initialize = [0])

    ####initialize parameters####

    def rank_param_initialize(model, k):
        return(r_k_dict.get(k))

    model.r_k = Param(model.K, initialize = rank_param_initialize)

    #model.r_k.pprint()

    def penalty_param_initialize(model, k):
        return(1/model.r_k[k])

    model.p_k = Param(model.K, initialize = penalty_param_initialize)

    #model.p_k.pprint()

    def discount_param_initialize(model, k, t):
        discount_temp = model.p_k[k]
        #time_diff = T-t
        return(1/((1+discount_temp)**(t-1)))

    model.tau_k_t = Param(model.K, model.T, initialize = discount_param_initialize)

    #model.tau_k_t.pprint()

    def quality_param_initialize(model, k, i):
        if q_k_i_dict.get(tuple([k,i])) == None:
            return(0)
        else:
            return(q_k_i_dict.get(tuple([k,i])))

    model.q_k_i = Param(model.K, model.I, initialize = quality_param_initialize)

    #model.q_k_i.pprint()

    def beg_inv_param_initialize(model, k, i):
        if z_k_i_init_dict.get(tuple([k,i])) == None:
            return(0)
        else:
            return(z_k_i_init_dict.get(tuple([k,i])))

    model.z_k_i_init = Param(model.K, model.I, initialize = beg_inv_param_initialize)

    #model.z_k_i_init.pprint()

    def unfulfilled_demand_param_initialize(model, k):
        return(alpha_k_init_dict.get(k))

    model.alpha_k_intialize = Param(model.K, initialize = unfulfilled_demand_param_initialize)

    #model.alpha_k_intialize.pprint()

    def delay_dist_param_initialize(model, k, i, t):
        if f_k_i_tDiff_dict.get(tuple([k,i,t])) == None:
            return(0)
        else:
            return(f_k_i_tDiff_dict.get(tuple([k,i,t])))

    model.f_k_i_tDiff = Param(model.K, model.I, model.T_beg, initialize = delay_dist_param_initialize)

    #model.f_k_i_tDiff.pprint()

    def incoming_orders_initialize(model, k, i, t):
        if o_k_i_t_dict.get(tuple([k,i,t])) == None:
            return(0)
        else:
            return(o_k_i_t_dict.get(tuple([k,i,t])))

    model.o_k_i_t = Param(model.K, model.I, model.T, initialize = incoming_orders_initialize)

    #model.o_k_i_t.pprint()

    def cost_param_initialize(model, k, i):
        if c_k_i_dict.get(tuple([k,i])) == None:
            return(0)
        else:
            return(c_k_i_dict.get(tuple([k,i])))

    model.c_k_i = Param(model.K, model.I, initialize = cost_param_initialize)

    #model.c_k_i.pprint()

    def budget_param_initialize(model):
        return(b)

    model.b = Param(initialize = budget_param_initialize)

    #model.b.pprint()

    def supplier_cap_param_initialize(model, k, i, t):
        if CAP_k_i_t_dict_mu_sd.get(tuple([k,i,t])) == None:
            return(0)
        else:
            temp = CAP_k_i_t_dict_mu_sd.get(tuple([k,i,t]))
            mean = temp[0]
            sd = CAP_reliability*temp[1]
            return(mean+sd)

    model.cap_k_i_t = Param(model.K, model.I, model.T, initialize = supplier_cap_param_initialize)

    #model.cap_k_i_t.pprint()

    #assume preparing for one sd above the mean
    def mu_plus_sigma_demand_param_initialize(model, k, t):
        temp = D_k_t_dict_mu_sd.get(tuple([k,t]))
        mean = temp[0]
        sd = DEMAND_reliability*temp[1]
        return(mean+sd)

    model.d_k_t = Param(model.K, model.T, initialize = mu_plus_sigma_demand_param_initialize)

    #model.d_k_t.pprint()

    ####initialize variables####
    model.x_k_i_t = Var(model.K, model.I, model.T, within = NonNegativeReals) #amount ordered
    model.y_k_i_t = Var(model.K, model.I, model.T, within = NonNegativeReals) #amount recieved
    model.z_k_i_t = Var(model.K, model.I, model.T, within = NonNegativeReals)
    model.alpha_k_t = Var(model.K, model.T, within = NonNegativeReals)
    model.beta_k_i_t = Var(model.K, model.I, model.T, within = NonNegativeReals)
    return(model)

In [6]:
def initialize_objective(model):
    model.Objective = Objective(expr = 
                                (sum(model.q_k_i[k,i]*sum(model.tau_k_t[k,t]*model.beta_k_i_t[k,i,t] 
                                                          for t in model.T) 
                                     for i in model.I for k in model.K)),
                                sense = maximize)
    return(model)

In [7]:
def initialize_constraints(model):
    def beggining_inventory_constraint_initialize(model, k, i):
        if (model.z_k_i_init[k,i] != None):
            return(model.z_k_i_t[k,i,1] == model.z_k_i_init[k,i])
        else:
            return(Constraint.Skip)
        
        model.beggining_inventory_constraint = Constraint(model.K, model.I, 
                                                          rule = beggining_inventory_constraint_initialize)
#model.beggining_inventory_constraint.pprint()

    def initalize_unsatisfied_demand_constraint_initialize(model, k):
        return(model.alpha_k_t[k,1] == model.alpha_k_intialize[k])

    model.initalize_unsatisfied_demand_constraint_initialize = \
    Constraint(model.K, rule = initalize_unsatisfied_demand_constraint_initialize)
    
    def incoming_orders_constraint_initialize(model, k, i, t):
        return(model.y_k_i_t[k,i,t] - 
               sum(model.f_k_i_tDiff[k, i, t-t_ordered_time]*model.x_k_i_t[k,i,t_ordered_time] 
                   for t_ordered_time in range(1,t)) == 0)#- model.o_k_i_t[k,i,t] == 0)
    
    model.incoming_ordered_constraint = Constraint(model.K, model.I, model.T, 
                                                   rule = incoming_orders_constraint_initialize)

#model.incoming_ordered_constraint.pprint()
    
    def calculate_available_inventory_constraint_initialize(model, k, i, t):
        if (t > 1):
            return(model.z_k_i_t[k,i,t] - model.z_k_i_t[k,i,t-1] - model.y_k_i_t[k,i,t-1] + model.beta_k_i_t[k,i,t-1] == 0)
        else:
            return(Constraint.Skip)

    model.calculate_available_inventory_constraint = Constraint(model.K, model.I, model.T,
                                                                rule = 
                                                                calculate_available_inventory_constraint_initialize)
    
    def cannot_fulfill_more_than_available_constraint_initialize(model, k,i,t):
        return(model.beta_k_i_t[k,i,t] - model.z_k_i_t[k,i,t] <= 0)

    model.cannot_fulfill_more_than_available_constraint = \
    Constraint(model.K, model.I, model.T, rule = cannot_fulfill_more_than_available_constraint_initialize)
    
    def calculate_unsatisfied_demand_constraint_initialize(model,k,t):
        if t > 1:
            return(model.alpha_k_t[k,t] - model.alpha_k_t[k,t-1] + sum(model.beta_k_i_t[k,i,t] for i in model.I) == \
                   model.d_k_t[k,t])
        else:
            return(Constraint.Skip)

    model.calculate_unsatisfied_demand_constraint = \
    Constraint(model.K, model.T, rule = calculate_unsatisfied_demand_constraint_initialize)
    
    #def warehouse_capacity_constraint_initialize(model, t):
    #    return(sum(model.s_k[k]*model.z_k_i_t[k,i,t] for k in model.K for i in model.I) <= model.h)

    #model.warehouse_capacity_constraint = \
    #Constraint(model.T, rule = warehouse_capacity_constraint_initialize)
    
    def budget_constraint_initialize(model):
        return(sum(model.c_k_i[k,i]*model.x_k_i_t[k,i,t] 
                   for k in model.K for i in model.I for t in model.T) 
               <= model.b)

    model.budget_constraint = \
    Constraint(rule = budget_constraint_initialize)
    
    def supplier_constraint_initialize(model, k, i, t):
        return(model.x_k_i_t[k,i,t] <= model.cap_k_i_t[k,i,t])

    model.supplier_constraint = \
    Constraint(model.K, model.I, model.T, rule = supplier_constraint_initialize)
    
    return(model)

In [8]:
def extract_data(model, reliability_level, budget):
    
    #extract from unsatisfied_df
    def unsatisfied_df_update(unsatisfied_df):
        if unsatisfied_df.empty:
            unsatisfied_df = pd.DataFrame(list(model.alpha_k_t.extract_values().items()),\
                                          columns = ['sets','units_unsatisfied'])
            unsatisfied_df.loc[:,'item_type']=unsatisfied_df.sets.map(lambda x:x[0])
            unsatisfied_df.loc[:,'time_interval']=unsatisfied_df.sets.map(lambda x:x[1])
            unsatisfied_df = unsatisfied_df[['item_type', 'time_interval', 'units_unsatisfied']]
            unsatisfied_df ['reliability_level'] = [reliability_level]*len(unsatisfied_df)
            unsatisfied_df['budget'] = [budget]*len(unsatisfied_df)
            return(unsatisfied_df)
        else:
            unsatisfied_df_temp = pd.DataFrame(list(model.alpha_k_t.extract_values().items()),\
                                          columns = ['sets','units_unsatisfied'])
            unsatisfied_df_temp.loc[:,'item_type']=unsatisfied_df_temp.sets.map(lambda x:x[0])
            unsatisfied_df_temp.loc[:,'time_interval']=unsatisfied_df_temp.sets.map(lambda x:x[1])
            unsatisfied_df_temp = unsatisfied_df_temp[['item_type', 'time_interval', 'units_unsatisfied']]
            unsatisfied_df_temp ['reliability_level'] = [reliability_level]*len(unsatisfied_df_temp)
            unsatisfied_df_temp['budget'] = [budget]*len(unsatisfied_df_temp)

            return(unsatisfied_df.append(unsatisfied_df_temp, ignore_index=True))
        
    #extract from satisfied_df
    def fulfilled_df_update(fulfilled_df):
        if fulfilled_df.empty:
            fulfilled_df = pd.DataFrame(list(model.beta_k_i_t.extract_values().items()),columns = ['sets','units_fulfilled'])
            fulfilled_df.loc[:,'item_type']=fulfilled_df.sets.map(lambda x:x[0])
            fulfilled_df.loc[:,'supplier']=fulfilled_df.sets.map(lambda x:x[1])
            fulfilled_df.loc[:,'time_interval']=fulfilled_df.sets.map(lambda x:x[2])
            fulfilled_df['reliability_level'] = [reliability_level]*len(fulfilled_df)
            fulfilled_df['budget'] = [budget]*len(fulfilled_df)
            return(fulfilled_df)
        else:
            fulfilled_df_temp = pd.DataFrame(list(model.beta_k_i_t.extract_values().items()),columns = ['sets','units_fulfilled'])
            fulfilled_df_temp.loc[:,'item_type']=fulfilled_df_temp.sets.map(lambda x:x[0])
            fulfilled_df_temp.loc[:,'supplier']=fulfilled_df_temp.sets.map(lambda x:x[1])
            fulfilled_df_temp.loc[:,'time_interval']=fulfilled_df_temp.sets.map(lambda x:x[2])
            fulfilled_df_temp['reliability_level'] = [reliability_level]*len(fulfilled_df_temp)
            fulfilled_df_temp['budget'] = [budget]*len(fulfilled_df_temp)

            return(fulfilled_df.append(fulfilled_df_temp, ignore_index=True))
        
    def ordered_df_update(ordered_df):
        if ordered_df.empty:
            ordered_df = pd.DataFrame(list(model.x_k_i_t.extract_values().items()),columns = ['sets','units_to_order'])
            ordered_df.loc[:,'item_type']=ordered_df.sets.map(lambda x:x[0])
            ordered_df.loc[:,'supplier']=ordered_df.sets.map(lambda x:x[1])
            ordered_df.loc[:,'time_interval']=ordered_df.sets.map(lambda x:x[2])
            ordered_df['reliability_level'] = [reliability_level]*len(ordered_df)
            ordered_df['budget'] = [budget]*len(ordered_df)
            
            return(ordered_df)
        else:
            ordered_df_temp = pd.DataFrame(list(model.x_k_i_t.extract_values().items()),columns = ['sets','units_to_order'])
            ordered_df_temp.loc[:,'item_type']=ordered_df_temp.sets.map(lambda x:x[0])
            ordered_df_temp.loc[:,'supplier']=ordered_df_temp.sets.map(lambda x:x[1])
            ordered_df_temp.loc[:,'time_interval']=ordered_df_temp.sets.map(lambda x:x[2])
            ordered_df_temp['reliability_level'] = [reliability_level]*len(ordered_df_temp)
            ordered_df_temp['budget'] = [budget]*len(ordered_df_temp)
            
            return(ordered_df.append(ordered_df_temp, ignore_index = True))
    
    global unsatisfied_df
    global fulfilled_df
    global ordered_df
    
    unsatisfied_df = unsatisfied_df_update(unsatisfied_df)
    fulfilled_df = fulfilled_df_update(fulfilled_df)
    ordered_df = ordered_df_update(ordered_df)
    
    return(unsatisfied_df, fulfilled_df, ordered_df)

In [9]:
fulfilled_df = pd.DataFrame()
unsatisfied_df = pd.DataFrame()
ordered_df = pd.DataFrame()

for r in reliability_levels:
    
    objective_itr = [0,1]
    b = 1
    while(objective_itr[b-1] < objective_itr[b]):    
        model = initialize_model(r, r, b)
        model = initialize_objective(model)
        model = initialize_constraints(model)
        opt = pyomo.opt.SolverFactory("glpk")
        results = opt.solve(model)
        opt.solve(model)
        
        #extract data
        trade_off_dict[tuple([r,b*10000])] = value(model.Objective) #model.Objective.value()
        objective_itr.append(value(model.Objective))#model.Objective.value())
        
        unsatisfied_df, fulfilled_df, ordered_df = extract_data(model, r, b*10000)
        
        b = b + 1

ERROR: evaluating object as numeric value: beta_k_i_t[1,1,1]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object beta_k_i_t[1,1,1]
ERROR: evaluating object as numeric value: Objective
        (object: <class 'pyomo.core.base.objective.SimpleObjective'>)
    No value for uninitialized NumericValue object beta_k_i_t[1,1,1]


ValueError: No value for uninitialized NumericValue object beta_k_i_t[1,1,1]

In [10]:
model.beta_k_i_t.pprint()

beta_k_i_t : Size=910, Index=beta_k_i_t_index
    Key         : Lower : Value : Upper : Fixed : Stale : Domain
      (1, 1, 1) :     0 :  None :  None : False :  True : NonNegativeReals
      (1, 1, 2) :     0 :  None :  None : False :  True : NonNegativeReals
      (1, 1, 3) :     0 :  None :  None : False :  True : NonNegativeReals
      (1, 1, 4) :     0 :  None :  None : False :  True : NonNegativeReals
      (1, 1, 5) :     0 :  None :  None : False :  True : NonNegativeReals
      (1, 1, 6) :     0 :  None :  None : False :  True : NonNegativeReals
      (1, 1, 7) :     0 :  None :  None : False :  True : NonNegativeReals
      (1, 1, 8) :     0 :  None :  None : False :  True : NonNegativeReals
      (1, 1, 9) :     0 :  None :  None : False :  True : NonNegativeReals
     (1, 1, 10) :     0 :  None :  None : False :  True : NonNegativeReals
      (1, 2, 1) :     0 :  None :  None : False :  True : NonNegativeReals
      (1, 2, 2) :     0 :  None :  None : False :  True : NonNeg

In [ ]:
with open('test.csv', 'w') as f:
    for key in trade_off_dict.keys():
        f.write("%s,%s\n"%(key,trade_off_dict[key]))

In [ ]:
budget = []
reliability = []
obj_value = []

for k in trade_off_dict.keys():
    obj_value.append(trade_off_dict.get(k))
    budget.append(k[1])
    if (k[0] == 2):
        reliability.append(r'$\theta^D > \theta^CAP$ = 0')
    elif (k[0] == 1):
        reliability.append(r'$\theta^D > \theta^CAP$ = 1')
    else:
        reliability.append(r'$\theta^D > \theta^CAP$ = 2')

In [ ]:
trade_off_df = pd.DataFrame([budget,reliability, obj_value]).transpose()
trade_off_df.columns = ['budget', 'reliability', 'obj_value']

In [ ]:
fulfilled_df_item_time = fulfilled_df.groupby(['item_type', 'time_interval', 
                      'reliability_level', 'budget'])['units_fulfilled'].sum().reset_index()

reliability_df = pd.merge(fulfilled_df_item_time[['item_type', 'time_interval', 'units_fulfilled',
                                                  'reliability_level', 'budget']], 
                          unsatisfied_df[['item_type', 'time_interval', 'units_unsatisfied',
                                          'reliability_level', 'budget']], 
                          how='left', on=['item_type','time_interval',
                                         'reliability_level', 'budget'])

reliability_df['total_number_of_open_requests'] = reliability_df['units_fulfilled'] +\
reliability_df['units_unsatisfied'] 

reliability_df['percent_unsatisfied'] = \
reliability_df['units_unsatisfied']/reliability_df['total_number_of_open_requests']

In [ ]:
reliability_df_grouped = \
reliability_df.groupby(['reliability_level', 'budget'])['percent_unsatisfied'].sum().reset_index()

In [ ]:
reliability_df_grouped['risk_of_understock'] = reliability_df_grouped['percent_unsatisfied']/(T*K)

In [ ]:
fig, ax = plot.subplots(figsize=(12,6))


ax.plot('budget', 'risk_of_understock', 
          data=reliability_df_grouped[reliability_df_grouped['reliability_level'] == 0],
         marker='o', color = 'red', label = r'$\theta^D = \theta^{CAP}$ = 0' + ' (low reliability)')
ax.plot('budget', 'risk_of_understock', 
          data=reliability_df_grouped[reliability_df_grouped['reliability_level'] == 1],
         marker='o', color = 'orange', label = r'$\theta^D = \theta^{CAP}$ = 1' + ' (medium reliability)')
ax.plot('budget', 'risk_of_understock', 
          data=reliability_df_grouped[reliability_df_grouped['reliability_level'] == 2],
         marker='o', color = 'green', label = r'$\theta^D = \theta^{CAP}$ = 2' + ' (high reliability)')

ax.set_xlabel('Budget (' + r'$b$' + ')', fontsize = 13)
ax.set_ylabel('Overall expected percent of unsatisfied demand', fontsize = 13)

ax.set(ylim=(0, 1), xlim=(0, max(reliability_df_grouped['budget'])))
ax.set_title(r'$\bf{Overall ~ Expected ~Percent ~of ~Unsatisfied ~Demand}$'+
             '\n'+ r'$\bf{Under ~Various~ Budgets~ (b) ~and ~Reliability~ Level~ Assumptions~ (\theta^D, \theta^{CAP})}$' +
             '\n \n \n where the expected percent of unsatisfied demand for each item '+ r'$(k)$' + ' during time interval '+ r'$(t)$' + ', represented by: \n'+\
              r'$\psi_{k,t} = \frac{\alpha_{k,t}}{\alpha_{k,t} + \sum_{i \in I} \beta_{k,i,t}} \forall k \in K, t \in T$' +
             '\n ...is averaged over all items '+  r'$(K)$'+ ' and time intervals '+  r'$(T)$'+ ' to obtain the:'\
             '\n Overall expected percent of unsatisfied demand = ' + r'$\frac{\sum_{t \in T} \sum_{k \in K} \psi_{k,t}}{K \times T}$',
             fontsize = 16)

plot.legend()

In [ ]:
#what if you selected a budget of 1,100,000 and a low, medium, high reliability?
for r_planned in reliability_levels:
    for r_actual in reliability_levels:
        model = initialize_model(r_actual, r_actual, 11)
        model = initialize_objective(model)
        model = initialize_constraints(model)
        
        #set ordering schedule
        ordered_df_budget = \
        ordered_df[(ordered_df['budget'] == 11*10000) & (ordered_df['reliability_level'] == r_planned)]
        
        def ordering_schedule_initialize(model, k, i, t):
            order_amt = ordered_df_budget[(ordered_df_budget['item_type'] == k) & \
                                          (ordered_df_budget['supplier'] == i) & \
                                          (ordered_df['time_interval'] == t)]['units_to_order']
            return(model.x_k_i_t[k,i,t] == order_amt.values[0])
    
        model.ordering_schedule_intialize = \
        Constraint(model.K, model.I, model.T, rule = ordering_schedule_initialize)

        opt = pyomo.opt.SolverFactory("glpk")
        results = opt.solve(model)
        opt.solve(model)

        #unsatisfied_df, fulfilled_df, ordered_df = extract_data(model, r_planned, 11*10000)